# Context-FID Score Presentation
## Necessary packages and functions call

- Context-FID score: A useful metric measures how well the the synthetic time series windows ”fit” into the local context of the time series

In [11]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from context_fid import Context_FID
from metric_utils import display_scores
from cross_correlation import CrossCorrelLoss

## Data Loading

Load original dataset and preprocess the loaded data.

In [12]:
iterations = 5
data_name = 'ETTh1'
ori_data = np.load(f'../generated_datasets/real_data_npy/{data_name}.npy')
#ori_data = np.load(f'../generated_datasets/real_data_npy/etth_norm_truth_24_train.npy')
fake_data = np.load(f'../generated_datasets/model_{data_name}_generate_uncond_64.npy')
#fake_data = np.load(f'../generated_datasets/ddpm_fake_{data_name}_64.npy')
#fake_data = np.load(f'../generated_datasets/diffwave5_{data_name}.npy')[:len(ori_data), :, :]
ori_data = ori_data[:len(fake_data), :, :]

## Context-FID Score

- The Frechet Inception distance-like score is based on unsupervised time series embeddings. It is able to score the fit of the fixed length synthetic samples into their context of (often much longer) true time series.

- The lowest scoring models correspond to the best performing models in downstream tasks

In [13]:
context_fid_score = []

for i in range(iterations):
    context_fid = Context_FID(ori_data[:], fake_data[:ori_data.shape[0]])
    context_fid_score.append(context_fid)
    print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')
      
display_scores(context_fid_score)

Iter 0:  context-fid = 0.18646640018594307 

Iter 1:  context-fid = 0.18586163176622614 

Iter 2:  context-fid = 0.17857914638476519 

Iter 3:  context-fid = 0.17932179400112397 

Iter 4:  context-fid = 0.17974523213743898 

Final Score:  0.18199484089509949 ± 0.004761413225045591


## Correlational Score

- The metric uses the absolute error of the auto-correlation estimator by real data and synthetic data as the metric to assess the temporal dependency.

- For d > 1, it uses the l1-norm of the difference between cross correlation matrices.

In [14]:
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

In [15]:
x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

Iter 0:  cross-correlation = 0.038773005008113745 

Iter 1:  cross-correlation = 0.03495507914146936 

Iter 2:  cross-correlation = 0.031182041926478554 

Iter 3:  cross-correlation = 0.03773915128705052 

Iter 4:  cross-correlation = 0.025987931894362038 

Final Score:  0.033727441851494844 ± 0.006494676883310888
